# <center>**ML School by REU Data Scinece Club**</center>

----
## <center> Неделя №7 </center>

На этой неделе вам предстоит написать Adaboost, основанный на Decision tree. Это последний алгоритм, который мы будем писать сами.

Импортируем нужные библиотеки

In [50]:
import numpy as np
import pandas as pd
import math

from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

Загрузим данные, мы будем пытаться предсказать космические тела - галактики и звезды(колонка class). Что обозначает каждая колонка нам не особо важно также как и класс. Данные уже обработаны. Если вам интересно почитать о них, то вот [ссылка](https://www.sdss.org/dr18/) на официальный сайт

In [2]:
df = pd.read_csv('galaxy_data.csv').drop('objid', axis=1)
df.sample(10)

,ra,dec,u,g,r,i,z,run,rerun,camcol,field,specobjid,class,redshift,plate,mjd,fiberid
26999,250.911166,23.574331,19.52053,19.35182,19.30574,19.06973,19.14445,3926,301,3,180,1592073373284850000,QSO,1.879539,1414,53135,185
58511,231.821972,38.887347,18.68969,17.19725,16.67180,16.50511,16.43782,3958,301,3,16,3277535113566510000,STAR,-0.000256,2911,54631,147
91982,251.365852,49.269975,19.41021,18.56163,18.19202,18.04569,17.99791,4679,301,5,127,7113673484606390000,STAR,-0.000386,6318,56186,865
21577,156.660823,37.774681,16.93023,15.48110,14.77802,14.43641,14.13299,3704,301,1,155,1607820576519380000,GALAXY,0.050919,1428,52998,129
633,242.529673,43.458183,19.06716,17.20977,16.27894,15.85819,15.53006,3225,301,4,203,1317451096435750000,GALAXY,0.112044,1170,52756,539
64211,101.783016,27.677375,16.60670,15.47498,15.18488,15.09547,15.11422,6585,301,2,121,3012971717619380000,STAR,0.000080,2676,54179,231
5508,318.220922,11.205286,18.50027,16.90464,16.27460,16.02399,15.86739,1739,301,5,30,2128055056967170000,STAR,0.000052,1890,53237,379
37730,121.872070,54.513970,18.56970,16.95687,16.10067,15.66996,15.30100,4294,301,3,107,2106613208302250000,GALAXY,0.102320,1871,53384,198
93528,14.262778,0.521989,19.34459,17.60751,16.94042,16.68151,16.57630,4263,301,5,211,3503916864013180000,STAR,-0.000110,3112,54802,423
44964,21.195467,31.941222,17.71389,16.79022,16.47753,16.35932,16.32762,8111,301,5,200,2298122020510000000,STAR,-0.001002,2041,53387,583


В наших данных три класса. Сейчас мы не будем решать задачу мультиклассовой классификации, поэтому просто удалим 1 класс и изменим обозначения на -1 и 1 

In [3]:
df['class'].value_counts()

GALAXY    51323
STAR      38096
QSO       10581
Name: class, dtype: int64

In [4]:
df = df[df['class'].isin(['GALAXY', 'STAR'])]
df['class'].replace({'GALAXY':-1, 'STAR':1}, inplace=True)
df

,ra,dec,u,g,r,i,z,run,rerun,camcol,field,specobjid,class,redshift,plate,mjd,fiberid
0,47.372545,0.820621,18.69254,17.13867,16.55555,16.34662,16.17639,4849,301,5,771,8168632633242440000,1,0.000115,7255,56597,832
1,116.303083,42.455980,18.47633,17.30546,17.24116,17.32780,17.37114,6573,301,6,220,9333948945297330000,1,-0.000093,8290,57364,868
2,172.756623,-8.785698,16.47714,15.31072,15.55971,15.72207,15.82471,5973,301,1,13,3221211255238850000,1,0.000165,2861,54583,42
3,201.224207,28.771290,18.63561,16.88346,16.09825,15.70987,15.43491,4649,301,3,121,2254061292459420000,-1,0.058155,2002,53471,35
4,212.817222,26.625225,18.88325,17.87948,17.47037,17.17441,17.05235,4649,301,3,191,2390305906828010000,-1,0.072210,2123,53793,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,332.714103,25.583064,19.53736,17.41563,16.41278,15.95257,15.58614,8116,301,3,67,6708188165264660000,-1,0.076411,5958,56213,278
99996,173.757382,36.441603,18.33687,17.30365,17.16037,17.14895,17.14419,4504,301,2,111,2265404129658560000,1,-0.000497,2012,53493,340
99997,205.426531,38.499053,17.50690,15.63152,15.22328,15.04469,15.28668,4382,301,4,97,2257446413900210000,-1,0.004587,2005,53472,62
99998,337.135144,-9.635967,19.33946,17.21436,16.29697,15.86745,15.51556,2576,301,2,105,811847537492257000,-1,0.084538,721,52228,268


Разделите данные на train и test указав random_state = 1, test_size=0.25

In [47]:
Columns = set(df.columns)
Columns.remove('class')
Columns = list(Columns)

In [48]:
X_train, X_test, y_train, y_test = train_test_split(df[Columns], df['class'], test_size=0.25, random_state=1)

# **Задание 1. Функции для оптимизации (4 балла)**

Реализуем несколько полезных функций. Начнем с функции подсчета ошибки дерева - error_rate

$$error\ rate = {\frac{количество\ ошибочно\ предсказанных\ значений}{общее\ количество\ значений}} $$

In [52]:
def error_rate(y_true, y_pred):
    n = 0
    
    for i in range(len(y_true)):
        if (y_true[i] != y_pred[i]):
            n += 1  
            
    error_rate = n / len(y_true)
    
    return error_rate

Теперь давайте реализуем подсчет alpha, функция будет определять вес конкретного дерева во всей модели.

$$alpha = 0.5 *{\log_e ({\frac{1 - error\ rate}{error\ rate}})}$$

In [51]:
def alpha(error_rate):

    alpha = 0.5 * math.log((1 - error_rate)/error_rate)
    
    return alpha

Проверим работу двух функций оптимизаций.

# **Задание 2. Adaboost (12 баллов)**

Теперь мы можем написать наш Adaboost. В нем разрешается использовать DescionTreeClassifier(для них параметры можете выбрать сами). В sklearn можно взять веса для модели и настраивать их, главное не забудьте про их нормализацию, чтобы они не были слишком большими

In [ ]:
class Adaboost():
    def __init__(self, n_estimators=50, learning_rate=1.0, base_classifier=DecisionTreeClassifier()):

        self.n_estimators = n_estimators # количество моделей в adaboost
        self.learning_rate = learning_rate # скорость обучение
        self.base_classifier = base_classifier # модель которую мы используем

        self.alpha = [] # тут будут храниться значения alpha для каждой модели
        self.classifiers = [] # тут будут храниться сами модели
    
    def train(self, X, y):
        
       # Задайте начальные веса

       # Теперь вам нужно сделать цикл, который проходиться по указаному количеству моделей.
       # Модели и alpha добавляйте в self.alpha и self.classifiers
       # При обучении каждой модели в метод fit вам нужно передавать sample_weight=weights

       # Весь алгоритм отлично описан в лекции
    
    def predict(self, X):
        
        n_samples = len(X)
        y_pred = np.zeros(n_samples)

        for alpha, classifier in zip(self.alpha, self.classifiers):
            y_pred += alpha * classifier.predict(X)

        return np.sign(y_pred)

Протестируйте как работает ваш алгоритм и оцените результат по метрике roc_auc. Она должна быть выше 0.9

In [ ]:
# Ваш код тут

# **Задание 3. Catboost (4 балла)**

Теперь мы можем попровать настоящего монстра среди классического ML - Catboost от яндекса. Его не нужно настраивать, хотя и можно(у него тоже есть GridSearch, но не из Sklearn). Оцените его результат по roc_auc. 

In [ ]:
!pip install catboost

In [ ]:
# Ваш код тут